### Imports

In [1]:
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

### Connect to the Neo4j database and instantiate an LLM 

In [2]:
openai_apikey = ""
url = "bolt://localhost:7687"
username = "neo4j"
password = ""
graph = Neo4jGraph(url=url, username=username, password=password)

llm = ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview", openai_api_key=openai_apikey)

### Instatiate the LLMGraphTransformed

Add allowed nodes and relationships as a parameter to the Transformer. 

In [3]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
    "Person", "Group", "Position", "Location", "Institution", "Event", "Initiative", "Document",
    "Task", "Meeting", "Product", "Service", "Deadline", "Department", "Campaign", "Policy", "Technology",
    "Issue", "Solution"
    ],
    allowed_relationships=[
    "is located at", "participates in", "makes requests", "responds to requests", "writes", "edits", 
    "comments on", "belongs to", "holds", "has applied for", "contacted", "has title", "visited", "located in",
    "is filled by", "works for", "is offered by", "employs", "affiliates with", "hires", "houses", 
    "sponsors", "hires for", "reports to", "collaborates with", "approves", "rejects", "attends", "hosts", 
    "schedules", "submits", "reviews", "requires", "assigns", "shares", "mentions", "relates to", "updates", 
    "follows up on", "leads"
    ]
)

### Provide Email Text
This is a naive approach that takes in one email and converts it to entities and relationships in the Neo4j graph. Ideally we want to read a set of emails and perform this operation for all of them.

In [4]:
text = """
Hi Emily,

I'll be there. I've started compiling the marketing data for the quarter and have some insights to share that could impact our overall strategy.

Best,
Tom
"""

### Build the Knowledge Graph (KG) 

The code below is for transforming text into a graph structure. Here, you are creating a list of Document objects. Each Document object is initialized with some content (text). The variable text should contain the textual data you want to process. The llm_transformer (which stands for Language Model Transformer) is used to convert the list of Document objects into a format suitable for graph representation. 
We are concerned about:
* Entity Recognition: Identifying entities within the text.
* Relationship Extraction: Determining the relationships between identified entities.
* Graph Construction: Constructing a graph where nodes represent entities and edges represent relationships between these entities.

In [5]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer_filtered.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

graph.add_graph_documents(graph_documents)

Nodes:[Node(id='Emily', type='Person'), Node(id='Tom', type='Person')]
Relationships:[]


### Observations

I noticed that no relationships were detected. This is because none of the "allowed_relationships" were present in this email text. 